# Running an simulaulation model of a queueing system without priorities

The simulation model of a queueing system supports various types of input and service distributions, any number of channels. It is possible to specify the maximum length of the queue. By default, it is an infinite queue. In Kendall's notation: GI/G/n/r and GI/G/n

To run the simulation model, it requires 4 steps:
- creating an instance of the simulation model
- specifying the source distribution
- specifying the service distribution
- running the model


#### Import the QueueingSystemSimulator class and create an instance of the system. 
When creating it, specify the number of channels *n* and the maximum queue length *r*. If not specified, the queue is considered unlimited by default.

In [1]:
from most_queue.sim.base import QsSim

n = 3
r = 100

qs = QsSim(n, buffer=r)

#### Setting the input stream.
The *set_sources* method takes two parameters:
- distribution parameters
- type of distribution

For exponential distribution, you only need to pass one parameter - the intensity of the incoming flow. For most distributions, parameters are specified as a list.

#### Supported types of distributions
| Distribution Type | Type | Parameters |
| --- | --- | --- |
| Exponential | 'M' | mu
| Hyper-exponential 2nd order | 'N' | [y1, mu1, mu2]
| Gamma distribution | 'Gamma' | [mu, alpha]
| Erlang | 'E'| [r, mu]
| Cox 2nd order | 'C' | [y1, mu1, mu2]
| Pareto | 'Pa'| [alpha, K]
| Deterministic | 'D' | [b]
| Uniform | 'Uniform'| [mean, half_interval]
| Normal | 'Norm'| [mean, standard_deviation]

In [2]:
l = 1.0
qs.set_sources(l, 'M')

#### Set the service time distribution. Calculate the service intensity through the utilization coefficient and the number of channels

In [3]:
ro = 0.8  # коэффициент загрузки
mu = l / (ro * n)  # интенсивность обслуживания
qs.set_servers(mu, 'M')

#### To run the simulation, you need to call the *run* method and pass the number of jobs

In [4]:
num_of_jobs = 1000000
sim_results = qs.run(num_of_jobs)

print(sim_results)

Start simulation


Job served:    | 1/100 [00:00<00:10,  9.81it/s]10000/1000000:   1%|          | 1/100 [00:00<00:10,  9.81it/s]10000/1000000:   2%|▏         | 2/100 [00:00<00:10,  9.41it/s]20000/1000000:   2%|▏         | 2/100 [00:00<00:10,  9.41it/s]20000/1000000:   3%|▎         | 3/100 [00:00<00:10,  9.41it/s]30000/1000000:   3%|▎         | 3/100 [00:00<00:10,  9.41it/s]30000/1000000:   4%|▍         | 4/100 [00:00<00:10,  9.27it/s]40000/1000000:   4%|▍         | 4/100 [00:00<00:10,  9.27it/s]40000/1000000:   5%|▌         | 5/100 [00:00<00:10,  8.78it/s]50000/1000000:   5%|▌         | 5/100 [00:00<00:10,  8.78it/s]50000/1000000:   6%|▌         | 6/100 [00:00<00:10,  8.81it/s]60000/1000000:   6%|▌         | 6/100 [00:00<00:10,  8.81it/s]60000/1000000:   7%|▋         | 7/100 [00:00<00:10,  8.66it/s]70000/1000000:   7%|▋         | 7/100 [00:00<00:10,  8.66it/s]70000/1000000:   8%|▊         | 8/100 [00:00<00:10,  8.59it/s]80000/1000000:   8%|▊         | 8/100 [00:00<00:10,  8.59it/s]80000/1000000:   9%|▉  

Simulation is finished

QueueResults(v=[4.964263583429662, 44.21060583166838, 561.6042309039163], w=[2.566897354100529, 20.428673741231894, 243.8604461989103], p=[np.float64(0.0566726294821901), np.float64(0.13454881998518567), np.float64(0.1620646605396959), np.float64(0.12989212795275273), np.float64(0.10448760982255922), np.float64(0.08362295192279254), np.float64(0.06627088231033351), np.float64(0.05273264722191611), np.float64(0.042223834827367704), np.float64(0.0337411466258683), np.float64(0.026659885620654838), np.float64(0.021523834068576605), np.float64(0.01707266888036521), np.float64(0.01345776564845522), np.float64(0.010973184798749359), np.float64(0.008830757068137966), np.float64(0.007098756603427032), np.float64(0.005494505600759689), np.float64(0.004308006888214237), np.float64(0.003599851595187637), np.float64(0.0029131228147121343), np.float64(0.002375102698972048), np.float64(0.0019169588713547584), np.float64(0.0015593023115546825), np.float64(0.0012058717063385478

#### After completing the simsimulation, you can obtain the initial moments of waiting and staying times, as well as the probabilities of system states.

#### Compare the results of simuimulation with the results of numerical calculation for the model M/M/n/r

In [6]:
from most_queue.theory.fifo.mmnr import MMnrCalc
from most_queue.general.tables import times_print, probs_print

mmnr = MMnrCalc(n=n, r=r)
mmnr.set_sources(l=l)
mmnr.set_servers(mu=mu)
calc_results = mmnr.run()
print(calc_results)

print(f"Simulation duration: {sim_results.duration:.5f} sec")
print(f"Calculation duration: {calc_results.duration:.5f} sec")

times_print(sim_results.w, calc_results.w)
probs_print(sim_results.p, calc_results.p, 10)

QueueResults(v=array([  4.98876403,  44.65617857, 570.04571475]), w=[2.5887640341427005, 20.710111207913737, 248.52122903740172], p=[0.05617977528682399, 0.1348314606883776, 0.16179775282605313, 0.12943820226084254, 0.10355056180867403, 0.08284044944693923, 0.06627235955755141, 0.05301788764604113, 0.04241431011683291, 0.03393144809346633, 0.02714515847477307, 0.021716126779818462, 0.01737290142385477, 0.013898321139083819, 0.011118656911267056, 0.008894925529013647, 0.007115940423210917, 0.005692752338568735, 0.004554201870854989, 0.0036433614966839916, 0.0029146891973471935, 0.002331751357877755, 0.0018654010863022046, 0.0014923208690417638, 0.0011938566952334113, 0.0009550853561867292, 0.0007640682849493834, 0.0006112546279595069, 0.0004890037023676056, 0.0003912029618940845, 0.00031296236951526763, 0.0002503698956122142, 0.00020029591648977136, 0.00016023673319181712, 0.0001281893865534537, 0.00010255150924276299, 8.204120739421042e-05, 6.563296591536833e-05, 5.250637273229466e-05,

#### Check for correctness of simulation results with the results of calculation for the model M/D/n

In [7]:
from most_queue.theory.fifo.m_d_n import MDn

qs = QsSim(n)

qs.set_sources(l, 'M')
qs.set_servers(1.0 / mu, 'D')

sim_results = qs.run(num_of_jobs)

mdn = MDn(n=n)
mdn.set_sources(l=l)
mdn.set_servers(b=1 / mu)

calc_results = mdn.run()

print(f"Simulation duration: {sim_results.duration:.5f} sec")
print(f"Calculation duration: {calc_results.duration:.5f} sec")


probs_print(sim_results.p, calc_results.p, 10)

Start simulation


Job served:    | 0/100 [00:00<?, ?it/s]10000/1000000:   1%|          | 1/100 [00:00<00:08, 12.34it/s]10000/1000000:   2%|▏         | 2/100 [00:00<00:08, 12.19it/s]20000/1000000:   2%|▏         | 2/100 [00:00<00:08, 12.19it/s]30000/1000000:   3%|▎         | 3/100 [00:00<00:07, 12.19it/s]30000/1000000:   4%|▍         | 4/100 [00:00<00:08, 11.81it/s]40000/1000000:   4%|▍         | 4/100 [00:00<00:08, 11.81it/s]50000/1000000:   5%|▌         | 5/100 [00:00<00:08, 11.81it/s]50000/1000000:   6%|▌         | 6/100 [00:00<00:08, 11.49it/s]60000/1000000:   6%|▌         | 6/100 [00:00<00:08, 11.49it/s]70000/1000000:   7%|▋         | 7/100 [00:00<00:08, 11.49it/s]70000/1000000:   8%|▊         | 8/100 [00:00<00:08, 11.32it/s]80000/1000000:   8%|▊         | 8/100 [00:00<00:08, 11.32it/s]90000/1000000:   9%|▉         | 9/100 [00:00<00:08, 11.32it/s]90000/1000000:  10%|█         | 10/100 [00:00<00:08, 11.21it/s]100000/1000000:  10%|█         | 10/100 [00:00<00:08, 11.21it/s]110000/1000000:  11%|█      

Simulation is finished

Simulation duration: 10.28692 sec
Calculation duration: 0.00061 sec
------------------------------------
      Probabilities of states       
------------------------------------
 #  |      Num      |      Sim      
------------------------------------
 0  |   0.049841    |   0.049384    
 1  |    0.13286    |    0.13279    
 2  |    0.18476    |    0.18411    
 3  |    0.18195    |    0.1819     
 4  |    0.14595    |    0.14611    
 5  |    0.10403    |    0.10408    
 6  |   0.069852    |   0.070024    
 7  |   0.045764    |   0.045686    
 8  |   0.029764    |   0.029372    
 9  |   0.019336    |   0.019112    
------------------------------------

